# Generating a list of Catalan word frequencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Gemini-generated csvs

These were generated with the following prompt:

> Genérame una lista de 50 palabras en orden de uso en la vida diaria. Crea cinco archivos csv para descargar delimitados por ";", cada uno con 10 palabras. Los campos de cada archivo serán: 1) card_id; 2) front; 3) back; 4) catalan; 5) español, 6) palabra; 7) categoría_gramatical; 8) categoría_de_uso; 9) audio_file_name. El front será de tipo cloze: tendrá la frase en catalán seguida de la frase en castellano con una cloze c1 para la palabra catalana y una cloze c2 para la palabra en castellano. El back se utilizará para clickar en el audio tras revelar la carta, así que debe contener [sound:nom_exacte_del_fitxer.mp3], y nom_exacte_del_fitxer.mp3 debe coincidir con el campo 7) audio_file_name. Los nombres de fichero serán la frase catalana (sacada del campo 4) en minúsculas y snake case. El campo 5 tendrá la frase en castellano.


In [2]:
from glob import glob

anki_csvs = []

for f in  glob("input_data/gemini_anki_csvs/*"):
    df = pd.read_csv(f, sep=";")
    anki_csvs.append(df)
    
anki_csvs = pd.concat(anki_csvs)
anki_csvs.head()

,card_id,front,back,catalan,español,palabra,categoría_gramatical,categoría_de_uso,audio_file_name
0,cat_esp_201,CAT: El meu {{c1::fill}} juga al parc.<br>ESP:...,[sound:el_meu_fill_juga_al_parc.mp3],El meu fill juga al parc.,Mi hijo juega en el parque.,fill,nom,Gent/Família,el_meu_fill_juga_al_parc.mp3
1,cat_esp_202,CAT: La meva {{c1::filla}} estudia molt.<br>ES...,[sound:la_meva_filla_estudia_molt.mp3],La meva filla estudia molt.,Mi hija estudia mucho.,filla,nom,Gent/Família,la_meva_filla_estudia_molt.mp3
2,cat_esp_203,CAT: Aquest {{c1::nen}} és simpàtic.<br>ESP: E...,[sound:aquest_nen_es_simpatic.mp3],Aquest nen és simpàtic.,Este niño es simpático.,nen,nom,Gent/Edat,aquest_nen_es_simpatic.mp3
3,cat_esp_204,CAT: La {{c1::nena}} riu contenta.<br>ESP: La ...,[sound:la_nena_riu_contenta.mp3],La nena riu contenta.,La niña ríe contenta.,nena,nom,Gent/Edat,la_nena_riu_contenta.mp3
4,cat_esp_205,CAT: Va a l'{{c1::escola}} cada dia.<br>ESP: V...,[sound:va_a_l_escola_cada_dia.mp3],Va a l'escola cada dia.,Va a la escuela cada día.,escola,nom,Educació/Llocs,va_a_l_escola_cada_dia.mp3


## Gemini generated word list

> Puedes generar una lista de las 5000 palabras más comunes en catalán? Soy castellanoparlante y quiero aprenderlo memorizando el vocabulario más útil para mí. Divídela en secciones de unas 1000 palabras para segmentar el aprendizaje, de más comunes a más infrecuentes. Las quiero descargar como fichero de texto.



In [3]:
gemini_word_list = pd.read_csv("input_data/gemini_word_list/catalan_word_list_0001-1108.txt", header=None)
gemini_word_list

,0
0,de
1,la
2,i
3,a
4,el
...,...
1103,flor tallada
1104,espelma
1105,encenedor
1106,llumins


## Viccionari word list

[List of words from Viccionari](https://ca.wiktionary.org/wiki/Viccionari:Llista_de_freq%C3%BC%C3%A8ncies)

In [4]:
import re

word = re.compile('^[0-9]+\. (.+) [0-9]+$')

viccionari_words = []

for f in sorted(glob('input_data/viccionari/*.txt')):
    with open(f) as file:
        for line in file:
            viccionari_words.append(re.search(word, line).groups()[0])
            
viccionari_words[:10]

['de', 'la', 'i', 'el', 'a', "l'", 'en', 'que', "d'", 'va']

In [5]:
len(viccionari_words)

8000

## Unification

In [6]:
all_words = pd.concat([gemini_word_list[0], anki_csvs['palabra'], pd.Series(viccionari_words)])

all_words.duplicated().sum()

1890

In [7]:
deduped = all_words.drop_duplicates().reset_index()[0]

deduped

0               de
1               la
2                i
3                a
4               el
           ...    
8363      incursió
8364       lliçons
8365        Llíria
8366    majoritari
8367     mercaders
Name: 0, Length: 8368, dtype: object

In [16]:
deduped.to_csv('final_catalan_words_list.txt', index=False, header=False)